# **IE 6600 : Data Visualization and Computation**

**Content**

1. Title page 

2. Introduction

3. Problem statement

4. Data used

5. Analysis flow

6. Analysis

7. Conclusion



# **Introduction**

E-commerce is growing at fast pace. In E-commerce applications/websites customers provide feedback/rating while returning order. This generates large size of raw data. When dealing with such velocity and volume of big data to hold customers it is important to do analysis of sales data at evry six months. Thus, understanding and then making sense out of these historical sales is an important problem to be looked at.

Problem definition: The purpose of the analysis is to focus on how product sales data to be utilized to derive business insights. Web dashboard is presented as ouput to suggests based on multiple features if following product will be sold or not. 


# **Problem statement**

#### **Problem Defination:** Given a dataset with records of sold and unsold products, objective is to build a binary classification model which learns from features and suggests if a particular product will be in sold category ot not.

#### This output is helpful to know if we should continue to sell or we should remeove a particular product from the inventory.

#### Prediction is binary classification model which multiple categorical and continuous features/inputs 


#### It then tries to predict the sold flag in binary form 1 or 0. 
 
1.  1 indicates - continue selling the product
2.  0 means - discard the product

Machine Learning Models used :
1. Random forest classification model
2. XGBoost classifier model



# **Data Used**

Attached is a set of products in which we are trying to determine which products we should continue to sell, and which products to remove from our inventory. 

The file contains BOTH historical sales data AND active inventory, which can be discerned with the column titled "File Type".

Each row in the file represents one product.It is important to note that we have MANY products in our inventory, and very few of them tend to sell (only about 10% sell each year) and many of the products only have a single sale in the course of a year

SKU_number: This is the unique identifier for each product.\
Order: Just a sequential counter.\
SoldFlag: 1 = sold in past 6 mos. 0 = Not sold\
MarketingType = Two categories of how we market the product.\
NewReleaseFlag = Any product that has had a future release (i.e., Release Number > 1)


The kaggle dataset file from internet with URL = https://www.kaggle.com/datasets/flenderson/sales-analysis/code



In [ ]:
## Import required libraries
import os
import re
from time import time
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
from operator import itemgetter
from typing import Dict, Tuple, List, Set


## plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio


## visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
!pip install bokeh
from bokeh.plotting import figure, output_notebook, show

  
## data preprocessing 
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

## machine learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import plot_roc_curve, confusion_matrix



## model evaluation
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

## warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SemesterTwo/6600/6600ComputationAndVisualization/Project 2/SalesKaggle3.csv')

#**Data Cleaning and Preprocessing**

In [ ]:
print (df.shape)

In [ ]:
print("Size of Data is : " + str(df.size))

In [ ]:
print("Data types of Dataset \n\n" + str(df.dtypes))



In [ ]:
print("Data Information")
df.info()

In [ ]:
print("Summary of data")

df.describe().round(2).T.style.background_gradient(
    vmin=-1, vmax=1, cmap=sns.color_palette("RdYlGn", as_cmap=True))

In [ ]:
df.isnull().sum()

In [ ]:
"""61 % null values found in sold count column"""
100*df.isna().sum()/df.shape[0]

In [ ]:
df.columns

In [ ]:
df['File_Type'].value_counts()

In [ ]:
df_active = df[df["File_Type"]=="Active"]
df_historical = df[df["File_Type"]=="Historical"]

In [ ]:
df_active["SoldFlag"].isna().sum()

#### The null values in sold flag and sold count are coming from Active inventory.

#### As the Active inventory does not have sold flag/labels they are not used for model training.

# **EDA**

In [ ]:
df_historical

In [ ]:
df_historical.shape

In [ ]:
df_historical.describe()

In [ ]:
df_historical.info()

In [ ]:
df_historical.isnull().sum()

#### This forms training dataset with multiple input features and target = SoldFlag.\
#### SoldFlag = if a given product is been sold for last 6 months or not.

#### Encoding categorical features using label encoder

In [ ]:
le = LabelEncoder()
df_historical["MarketingType"] = le.fit_transform(df_historical["MarketingType"])

In [ ]:
df_historical = df_historical.drop(["Order","SKU_number", "File_Type"], axis=1)

In [ ]:
## shuffle the Dataset
df_historical = df_historical.sample(frac=1.0, random_state=1)

#### corelated features

In [ ]:
!pip install biokit

In [ ]:
%pylab inline
import pandas as pd
matplotlib.rcParams['figure.dpi'] = 145
matplotlib.rcParams['figure.figsize'] = (8,6)
from biokit.viz import corrplot
c = corrplot.Corrplot(df_historical)
c.plot(colorbar=False, method='square', shrink=.9 ,rotation=90, upper='circle',grid='grey',
       fontsize=6,label_color='purple',
       cmap='RdYlGn')
plt.show();


matplotlib.rcParams['figure.dpi'] = 72.5

# Calculate pairwise-correlation
matrix = df_historical.corr().round(2)

# Create a mask
mask = np.tril(np.ones_like(matrix, dtype=bool))

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(14, 11))
sns.heatmap(matrix, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap='RdYlGn')

plt.show()

#### Observation :- from corelation matrix, it is clear that soldFlag is highly corelated to soldCount > 0.6

#### Thus soldCount can be dropped as soldFlag function of soldCount, it is redundant.

In [ ]:
df_historical = df_historical.drop(["SoldCount"], axis=1)

In [ ]:
## divide the fields into categorical and numerical
categ_cols = ["ReleaseYear", "MarketingType", "New_Release_Flag","SoldFlag"]
num_cols = ["ReleaseNumber", "StrengthFactor", "PriceReg", "ItemCount", "LowUserPrice", "LowNetPrice"]

In [ ]:
## converting sold flag to 0 or 1 string column instead of float for discrete colour visualization
#df_historical["SoldFlag"] = df_historical["SoldFlag"].astype(str)

#  **Send code to plotly chart_studio**

In [ ]:
!pip install chart_studio
import chart_studio

In [ ]:
## plot the distributions category wise based on the target. 

import chart_studio
chart_studio.tools.set_credentials_file(username="damle.g",api_key="2AJTjwjFd1mrsqa4zVbs")


import chart_studio.plotly as py
import chart_studio.tools as tls

target = 'SoldFlag'
for i in categ_cols : 
  if i != target :
      df = df_historical.value_counts([i,target]).reset_index()
      df = df.rename(columns={0: "frequency"})
      fig = px.bar(df, x=i, y="frequency", color = target, width=800, height=400,
      title="Distribution of "+ i,template="ggplot2")
      py.plot(fig, filename = 'Distribution of categorical feature'+i, auto_open=False)
      fig.show() 




Observation:

1. ReleaseYear - 21st century seem to have more products released
2. Marketing type 1 is with less sold products than 0.
3. Most products have new releases with flag value 1 and out of them very few actually got sold

In [ ]:
import chart_studio
chart_studio.tools.set_credentials_file(username="gauri54dam",api_key="0blXm8AnpIJXEQO2qw6a")


import chart_studio.plotly as py
import chart_studio.tools as tls


## plot distribution and box plots numerical wise based on the target.
for i in num_cols : 

    fig = make_subplots(rows=1, cols=2)

    fig.add_trace(
    go.Histogram(x = df_historical[i],name = "Histogram"),
    row=1, col=1)
   
    fig.add_trace(
    go.Box(y = df_historical[i],name = "Boxplot"),
    row=1, col=2)
    
    fig.update_layout(height=400, width=800, title_text="Distribution of "+i, template="ggplot2")
    
    #py.plot(fig, filename = "Distribution of numerical features"+i, auto_open=False)
    fig.show()


#### Observation



1.   Release no - Most of the products have their release numbers < 20.
2.   PriceReg - Most of the products fall under value of 1000.
3.   ItemCount - Item count has some outliers > 1400 but most of values < 800.

LowUserPrice and LowNetPrice are with outliers and distribution is right skewed.
StrengthFactor is having high values as shown in scatter plot below:




In [ ]:
fig = px.histogram(df_historical["StrengthFactor"])
fig.update_layout(height=400, width=1000, title_text="Distribution of StrengthFactor", template="ggplot2")
fig.show()



This value has high range maximum > 6000, this can be scaled before feeding it to ML model.

# **Feature Scaling**

#### For this classification, Random forest and XGboost are used.
#### These decision trees do not require normalization of their inputs; and since XGBoost is essentially an ensemble algorithm comprised of decision trees, it does not require normalization for the inputs either

# **Removing Class Imbalance**

In [ ]:

df_historical.SoldFlag.value_counts()

#### There are more number of 0 labels than 1 positive labels.

#### Only 17% of data is positively labeled

#### This effects accuracy of model thus f1 score is used which takes in account this imbalance

In [ ]:
fig = plt.figure(figsize=(14, 7))
fig.set_dpi=300

plt.title("Sold Flag distribution")
sns.histplot(df_historical.SoldFlag)

plt.legend(title="Sold flag", loc='upper right')
plt.show()

In [ ]:
df_historical_sold_flag_0 = df_historical[df_historical['SoldFlag']==0].sample(n=12996)

In [ ]:
df_historical_sold_flag_1 = df_historical[df_historical['SoldFlag']==1]

In [ ]:
df_historical_sold_flag_0.shape

In [ ]:
df_historical_sold_flag_1.shape

In [ ]:
df_historical_new = pd.concat([df_historical_sold_flag_0, df_historical_sold_flag_1], axis=0)

In [ ]:
## shuffle the Dataset
df_historical_new = df_historical_new.sample(frac=1.0, random_state=1)

In [ ]:
df_historical_new.shape

In [ ]:
fig = plt.figure(figsize=(14, 7))
fig.set_dpi=300

plt.title("Sold Flag distribution")
sns.histplot(df_historical_new.SoldFlag)

plt.legend(title="Sold flag", loc='upper right')
plt.show()

In [ ]:
df_historical_new

# **Define features and labels**



In [ ]:
y = df_historical['SoldFlag']

In [ ]:
x = df_historical.drop('SoldFlag', axis=1)

In [ ]:
x.shape

In [ ]:
y.shape

# **Important Functions**





In [ ]:
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    endy = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    f1score = round(f1_score(labels, pred), 3)
    print('{} -- Accuracy: {} | Precision: {} | Recall: {} | F1Score: {} | Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   f1score,
                                                                                   round((endy - start)*1000, 1)))

In [ ]:
### k fold cross validation to choose hyper parameters correctly.

def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

# **Train Test and Validation split**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
## splitting test dataset in half one for validation tests and other for actual test
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=1)

In [ ]:
for dataset in [y_train, y_val, y_test]:
  print(round(len(dataset)/len(y), 2))

In [ ]:
for dataset in [x_train, x_val, x_test]:
  print(len(dataset))

In [ ]:
for dataset in [y_train, y_val, y_test]:
  print(len(dataset))

In [ ]:
print(x_train.shape)

In [ ]:
print(x_val.shape)

In [ ]:
print(x_test.shape)

# **Random Forest**

In [ ]:
rf = RandomForestClassifier()
parameters = {
      'n_estimators': [5, 50, 250],
      'max_depth': [2, 4, 8, 16, 32, None]
}

rf_model = GridSearchCV(rf, parameters, cv=5)
rf_model.fit(x_train, y_train)

print_results(rf_model)

#### Hyper parameter tuning with 5 fold cross validation using GridsearchCV

In [ ]:
rf_model.best_estimator_

In [ ]:
evaluate_model('RF', rf_model, x_val, y_val)

In [ ]:
evaluate_model('RF', rf_model, x_test, y_test)

In [ ]:
cf_matrix_rf = confusion_matrix(y_test, rf_model.predict(x_test))

In [ ]:
import chart_studio
chart_studio.tools.set_credentials_file(username="gauri54dam",api_key="0blXm8AnpIJXEQO2qw6a")


import chart_studio.plotly as py
import chart_studio.tools as tls
pio.templates.default = "simple_white"

px.defaults.template = "ggplot2"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
px.defaults.width = 600
px.defaults.height = 400

fig = px.density_heatmap(cf_matrix_rf, x=y_test, y= rf_model.predict(x_test),  template="seaborn")
py.plot(fig, filename = "Confusion matrix", auto_open=False)
fig.show()

In [ ]:
plot_roc_curve(rf_model, x_train, y_train)
plt.title("Train ROC")
plt.show()
plot_roc_curve(rf_model, x_val, y_val)
plt.title("Validation ROC")
plt.show()
plot_roc_curve(rf_model, x_test, y_test)
plt.title("Test ROC")
plt.show()

# **XGboost**

In [ ]:
gb = xgb.XGBClassifier(objective='binary:logistic')
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100]
}

boosting_model = GridSearchCV(gb, parameters, cv=5)
boosting_model.fit(x_train, y_train)

print_results(boosting_model)

In [ ]:
boosting_model.best_estimator_

In [ ]:
evaluate_model('Boosting', boosting_model, x_val, y_val)

In [ ]:
evaluate_model('Boosting', boosting_model, x_test, y_test)

In [ ]:
cf_matrix_boosting = confusion_matrix(y_test, boosting_model.predict(x_test))

In [ ]:
import chart_studio
chart_studio.tools.set_credentials_file(username="gauri54dam",api_key="0blXm8AnpIJXEQO2qw6a")


import chart_studio.plotly as py
import chart_studio.tools as tls


pio.templates.default = "simple_white"

px.defaults.template = "ggplot2"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
px.defaults.width = 600
px.defaults.height = 400

fig = px.density_heatmap(cf_matrix_boosting, x=y_test, y= boosting_model.predict(x_test),  template="seaborn")
py.plot(fig, filename = "Confusion matrix", auto_open=False)
fig.show()

In [ ]:
plot_roc_curve(boosting_model, x_train, y_train)
plt.title("Train ROC")
plt.show()
plot_roc_curve(boosting_model, x_val, y_val)
plt.title("Validation ROC")
plt.show()
plot_roc_curve(boosting_model, x_test, y_test)
plt.title("Test ROC")
plt.show()

# **Information Extraction and Model comparison**

The Model accuracy comparison:


1. Random Forest - 83%
2. XGboost - 84%



---




Random Forest

RF is ensemble model which combines the outputs of multiple trees running model independently.
RF models unline SVM and LR can be used for both categorical and continuous target variables. It trains fast but predictions is slow. 

It gives best accuracy with complex and messy data. 

Using gridsearch cv the best performing hyperparameters are selected. **`RandomForestClassifier(max_depth=8, n_estimators=250)`**



---

Boosting - XG boosting tree.

Boosting and RF are ensemble models which combine the outputs of multiple models.

It can handle complex models like RF. It has faster prediction time so faster than RF. But as each tree is sequentially learning from previous model mistakes it has slow trainigng time.

Using gridsearch cv the best performing hyperparameters are selected.
**`XGBClassifier(n_estimators=250) `**

---




Using hyperparameter tuning of models RF and boosting it can be seen that Random Forest does better with few, deep trees, and Boosted Trees does better with many, shallow trees.


# **Conclusion**

A retail firm which has many products in their inventory, but very few of them tend to sell (only about 10% sell each year) and many of the products only have a single sale in the course of a year.

From the classifier, the sales, growth and analytics team of the retail firm can determine which products from their inventory should they retain to sell and the ones to discard.

